# US Treasury Yield Prediction

Reference: https://ieeexplore.ieee.org/document/9049511 <br>
Data: https://home.treasury.gov/ <br>
Domain: Multi Dimensional Multistep Forecasting<br><br>
We use Deep Neural Networks to predict the yield curve for next 10 days based on the observations of last 30 days. Specifically we use an Long Short Term Memory (LSTM) based Encoder Decoder network architecture with Attention mechanism.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn.modules import dropout

In [ ]:
plt.style.use("Solarize_Light2")
plt.rcParams["figure.figsize"] = (10, 7)
# plt.figure(dpi=300)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/Projects/Deep Bonds/data/"

In [ ]:
yield_df = pd.read_csv(data_path + "US_Treasury_Yield.csv")
yield_df["Date"] = yield_df["Date"].apply(lambda  x: datetime.datetime.strptime(str(x), "%m/%d/%y"))
yield_df = yield_df.sort_values(["Date"])
yield_df.drop(["2 Mo"], axis=1, inplace=True)

In [ ]:
yield_df.iloc[5000:5005]

In [ ]:
yield_df.describe()

In [ ]:
main_df = yield_df.copy()
main_df = main_df.sort_values(["Date"])

Missing Data is handled with interpolation in the Yield Curve. That is the missing values are replaced by the adjacent term rate.

In [ ]:
# main_df.iloc[:, 1:] = main_df.iloc[:, 1:].interpolate(axis=1, limit_direction="both")
for c in list(main_df.columns)[1:]:
  main_df.loc[main_df[c].isna(), c] = main_df.loc[main_df[c].isna(), c].fillna(main_df[c].mean())
# main_df.fillna(0, inplace=True)

In [ ]:
ridx = np.random.randint(0, len(yield_df), 1)
main_df.iloc[ridx]

In [ ]:

plt.plot(np.array(yield_df.iloc[ridx].drop(["Date"], axis=1)).squeeze(), ">")
plt.xticks(list(range(12)), list(main_df.columns)[1:])
plt.title("Yield Curve for: {}".format(list(main_df.iloc[ridx]["Date"])[0].date()))
plt.ylabel("Rate")
plt.legend()
plt.show()

In [ ]:
# ridx = np.random.randint(0, len(yield_df), 1)
plt.plot(np.array(main_df.iloc[ridx].drop(["Date"], axis=1)).squeeze(), "r<", label="Interpolated")
plt.plot(np.array(yield_df.iloc[ridx].drop(["Date"], axis=1)).squeeze(), ">", label="Actual")
plt.xticks(list(range(12)), list(main_df.columns)[1:])
plt.title("Yield Curve for: {}".format(list(main_df.iloc[ridx]["Date"])[0].date()))
plt.ylabel("Rate")
plt.legend()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
for c in list(yield_df.columns)[1:]:
  plt.plot(list(yield_df["Date"]), yield_df[c], "-", alpha=0.8, label=c)
plt.legend()
plt.title("Evolution of Term Structure")
plt.show()

Transforming the data for traning using a Minmax Scaler which assigns a value of 1 to the maximum value and 0 to the minimum.

In [ ]:
scaler = MinMaxScaler()
main_df.iloc[:, 1:] = scaler.fit_transform(main_df.iloc[:, 1:])

In [ ]:
main_df.head()

In [ ]:
look_back = 30
look_ahead = 10

In [ ]:
X_data = []
y_data = []

for i in range(look_back, len(main_df)):
  if i + look_ahead < len(main_df):
    _x, _y = main_df.iloc[i-look_back: i], main_df.iloc[i: i + look_ahead]
    _x, _y = np.array(_x.drop(["Date"], axis=1)), np.array(_y.drop(["Date"], axis=1))
    X_data.append(_x), y_data.append(_y)


In [ ]:
X_tensor = torch.from_numpy(np.array(X_data)).float()
y_tensor = torch.from_numpy(np.array(y_data)).float()

In [ ]:
dataset = TensorDataset(X_tensor, y_tensor)
dl = y_tensor.shape[0]
batch_size = 32

train, valid, test = random_split(dataset, [int(0.75*dl), int(0.15*dl), dl - int(0.75*dl) - int(0.15*dl)])
train_dl = DataLoader(train, shuffle=True, batch_size=batch_size, drop_last=True)
valid_dl = DataLoader(valid, shuffle=True, batch_size=batch_size, drop_last=True)
test_dl = DataLoader(test, shuffle=False, batch_size=batch_size, drop_last=True)

In [ ]:
class EncoderModel(nn.Module):
  def __init__(self, input_dim, batch_size, hidden_dim, n_layers):
    super(EncoderModel, self).__init__()
    self.input_dim = input_dim
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size

    self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=self.hidden_dim, 
                        batch_first=True, num_layers=self.n_layers)
    
  def forward(self, input_seq, hidden):
    output, hidden = self.lstm(input_seq, hidden)
    return output, hidden

In [ ]:
class AttnEncoderModel(nn.Module):
  def __init__(self, input_dim, batch_size, hidden_dim, n_layers, input_seq=look_back, output_seq=look_ahead):
    super(AttnEncoderModel, self).__init__()
    self.input_dim = input_dim
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size

    self.input_seq = input_seq
    self.output_seq = output_seq

    self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=self.hidden_dim, 
                        batch_first=True, num_layers=self.n_layers, dropout=0.2)
    
    self.attn = nn.Linear(self.hidden_dim, 1)
    
  def forward(self, input_seq, hidden):
    output, hidden = self.lstm(input_seq, hidden)
    o1 = self.attn(output).squeeze(-1)
    weights = nn.Softmax(dim=1)(o1)
    weights = weights.repeat(1, 1, self.hidden_dim).view(self.batch_size, self.hidden_dim, self.input_seq)
    attn_mul = weights.transpose(2, 1) * output
    attn_mul = attn_mul.sum(dim=1)
    output = attn_mul.repeat(1, self.output_seq).reshape(-1, self.output_seq, self.hidden_dim)
    return output, hidden

In [ ]:
class DecoderModel(nn.Module):
  def __init__(self, input_dim, batch_size, hidden_dim, n_layers):
    super(DecoderModel, self).__init__()
    self.input_dim = input_dim
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size
    
    self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=self.hidden_dim, 
                        batch_first=True, num_layers=self.n_layers, dropout=0.2)
    
  def forward(self, enco, hidden):
    output, hidden = self.lstm(enco, hidden)
    return output, hidden

In [ ]:
class EncDecModel(nn.Module):
  def __init__(self, encoder, decoder, output_seq, output_dim):
    super(EncDecModel, self).__init__()
    self.encoder = encoder
    self.decoder = decoder

    self.output_dim = output_dim
    self.output_seq = output_seq

  def forward(self, input_seq, hidden):
    out1, hidden = self.encoder(input_seq, hidden)
    out1 = out1[:, -self.output_seq:, :]
    out1 = nn.ReLU()(out1)
    out2, hidden = self.decoder(out1, hidden)
    out2 = out2[:, :, -self.output_dim:]
    # out2 = nn.ReLU()(out2)
    return out2, hidden

In [ ]:
input_dim = 11
input_seq = look_back

output_seq = look_ahead
output_dim = input_dim

enc_hidden = 300
dec_hidden = enc_hidden

n_layers = 4

# enc_model = EncoderModel(input_dim=input_dim, hidden_dim=enc_hidden, 
#                         n_layers=n_layers, batch_size=batch_size)

enc_model = AttnEncoderModel(input_dim=input_dim, hidden_dim=enc_hidden, 
                        n_layers=n_layers, batch_size=batch_size, 
                        input_seq=input_seq, output_seq=output_seq)

dec_model = DecoderModel(input_dim=enc_hidden, hidden_dim=dec_hidden,
                        n_layers=n_layers, batch_size=batch_size)

model = EncDecModel(encoder=enc_model, decoder=dec_model, 
                            output_seq=output_seq, output_dim=output_dim)

In [ ]:
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.HuberLoss()
n_epochs = 150

In [ ]:
patience=3
train_losses = []
valid_losses = []

for epoch in range(n_epochs):
  # if patience == 0:
  #     print("Early Stopping the training.\nEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}").format(epoch, train_loss, valid_loss)
  #     break

  train_loss = 0.0
  valid_loss = 0.0  
  
  hidden = (torch.zeros(n_layers, batch_size, enc_hidden).requires_grad_().cuda(),
            torch.zeros(n_layers, batch_size, enc_hidden).requires_grad_().cuda())

  model.train()
  for xt, yt in train_dl:
    xt, yt = xt.cuda(), yt.cuda()

    optimizer.zero_grad()
    hidden = tuple([each.data for each in hidden])
    out, hidden = model(xt, hidden)
    loss = criterion(out, yt)
    loss.backward()
    optimizer.step()
    train_loss += loss.item() * xt.size(0)
  
  hidden = (torch.zeros(n_layers, batch_size, enc_hidden).cuda(),
            torch.zeros(n_layers, batch_size, enc_hidden).cuda())

  model.eval()
  for xv, yv in valid_dl:
    xv, yv = xv.cuda(), yv.cuda()
    with torch.no_grad():
      outv, _ = model(xv, hidden)
      lossv = criterion(outv, yv)
      valid_loss += lossv.item() * xv.size(0)
  
  train_loss = train_loss/len(train_dl.sampler)
  valid_loss = valid_loss/len(valid_dl.sampler)

  if epoch%5 == 0:
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
  
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  if epoch > 5 and valid_loss > valid_losses[-5]:
    patience -= 1


In [ ]:
plt.figure()
plt.plot(train_losses, "r", label="Train Error")
plt.plot(valid_losses, label="Validation Error")
plt.legend()
plt.title("Loss Comparision")
plt.show()

In [ ]:
outs = []
truey = []

hidden = (torch.zeros(n_layers, batch_size, enc_hidden).requires_grad_().cuda(),
            torch.zeros(n_layers, batch_size, enc_hidden).requires_grad_().cuda())

model.eval()
with torch.no_grad():
  for xte, yte in test_dl:
    xte, yte = xte.cuda(), yte.cuda()
    outte, _ = model(xte, hidden)
    outs.append(outte)
    truey.append(yte)

In [ ]:
otte = torch.concat(outs)
trye = torch.concat(truey)

In [ ]:
for _ in range(5):
  random_idx = np.random.randint(0, len(otte), 1)[0]
  fig, ax = plt.subplots(1, 2, figsize=(15, 4), sharey=True)
  plt.figure(dpi=50)
  plt.setp(ax, xticks=list(range(12)),xticklabels = list(main_df.columns)[1:])
  for i in range(look_ahead):
    r = scaler.inverse_transform(otte[random_idx, i, :].cpu().numpy().reshape(1, -1)).squeeze()
    ax[1].plot(r, "-",label=f"Day t+{i+1}")
  ax[1].legend()
  ax[1].set_title(f"Predicted Yield Curve next {look_ahead} days")
  # print("\n")

  for i in range(look_ahead):
    r = scaler.inverse_transform(trye[random_idx, i, :].cpu().numpy().reshape(1, -1)).squeeze()
    ax[0].plot(r, "-",label=f"Day t+{i+1}")
  ax[0].legend()
  ax[0].set_title(f"True Yield Curve next {look_ahead} days")
  plt.show()
  print("\n")

In [ ]:
y_preds = torch.from_numpy(scaler.inverse_transform(otte.view(-1, 11).cpu().numpy())).float().view(-1, look_ahead, 11)
y_trues = torch.from_numpy(scaler.inverse_transform(trye.view(-1, 11).cpu().numpy())).float().view(-1, look_ahead, 11)
# y_preds = scaler.inverse_transform(otte.view(-1, 11).cpu().numpy()).reshape(-1, look_ahead, 11)
# y_trues = scaler.inverse_transform(trye.view(-1, 11).cpu().numpy()).reshape(-1, look_ahead, 11)

print("\nAverage Absoulte Error for Day")
for i in range(look_ahead):
  l = nn.L1Loss()(y_preds[:, i, :], y_trues[:, i, :]).item()
  # l = np.abs(y_trues[:, i, :] - y_preds[:, i, :])/(y_trues[:, i, :])
  # l = np.round(l.reshape(-1), 5).mean(dtype=np.float64)
  # l = l.mean(dtype=np.float64)
  print(f"Day t+{i+1} :", l)

In [ ]:
print("Average Absoulte Error for Term:")
for i in range(11):
  l = nn.L1Loss()(y_preds[:, :, i], y_trues[:, :, i]).item()
  print(list(main_df.columns)[1:][i], ":",l)